In [99]:
import os
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, Model
import pathlib
from tensorflow.keras import Input
import numpy as np
from tensorflow.keras.layers import Lambda, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.metrics import IoU, BinaryIoU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K

In [97]:
def process_path(image_path, mask_path):
    image = tf.io.read_file(image_path)
    mask = tf.io.read_file(mask_path)
    image = tf.image.decode_png(image, channels = 3)
    mask = tf.image.decode_png(mask, channels = 1) / 255
    return image, mask

In [98]:
def batch_data (path_X, path_y, batch_size):
    X_names = os.listdir(path_X)
    X_path = [f'{path_X}/{file}' for file in X_names]
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    ds_train = tf.data.Dataset.from_tensor_slices((X_path, y_path))
    return ds_train.map(process_path).batch(batch_size)

In [17]:
os.listdir(path_X)

['kitsap27.tif_1888_4720.png',
 'austin19.tif_472_4484.png',
 'tyrol-w20.tif_4248_1888.png',
 'kitsap36.tif_3068_472.png',
 'chicago22.tif_3304_4484.png',
 'vienna10.tif_472_1180.png',
 'chicago36.tif_0_3776.png',
 'vienna36.tif_3068_4484.png',
 'kitsap13.tif_4248_2596.png',
 'kitsap13.tif_3068_1652.png',
 'austin11.tif_0_1416.png',
 'vienna7.tif_2832_4012.png',
 'kitsap6.tif_3776_1180.png',
 'austin17.tif_1180_1652.png',
 'vienna7.tif_472_1888.png',
 'chicago19.tif_2360_4012.png',
 'austin13.tif_1416_1416.png',
 'vienna10.tif_1180_3540.png',
 'austin11.tif_1416_944.png',
 'kitsap6.tif_4012_2596.png',
 'tyrol-w19.tif_4720_708.png',
 'kitsap13.tif_3540_0.png',
 'austin17.tif_3540_2832.png',
 'kitsap27.tif_2596_3776.png',
 'kitsap36.tif_0_3540.png',
 'austin17.tif_1652_472.png',
 'tyrol-w14.tif_3540_0.png',
 'austin13.tif_1416_4012.png',
 'chicago25.tif_944_4248.png',
 'chicago25.tif_2832_2832.png',
 'tyrol-w19.tif_2832_4012.png',
 'chicago22.tif_3068_708.png',
 'chicago19.tif_3068_3304.

In [24]:
home = os.environ['HOME']

In [27]:
path_X = os.path.join(home,'raw_data/small_dataset/sample_images')
path_y = os.path.join(home,'raw_data/small_dataset/sample_masks')
dataset = batch_data(path_X=path_X, path_y=path_y, batch_size=32)

In [28]:
def IoULoss(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    intersection = K.sum(K.dot(targets, inputs))
    total = K.sum(targets) + K.sum(inputs)
    union = total - intersection
    
    IoU = (intersection + smooth) / (union + smooth)
    return 1 - IoU

In [100]:
def unet_model(img_height, img_width, channels):

    #Input
    inputs = Input((img_height, img_width, channels))
    inputs = Lambda(lambda x: x / 255)(inputs) #Normalize the pixels by dividing by 255

    #Encoder where we are extracting the features    
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    convolution1 = BatchNormalization()(convolution1) 
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution1)
    pooling1 = MaxPooling2D((2, 2))(convolution1)
    
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(pooling1)
    convolution2 = BatchNormalization()(convolution2)
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution2)
    pooling2 = MaxPooling2D((2, 2))(convolution2)
     
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pooling2)
    convolution3 = BatchNormalization()(convolution3)
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution3)
    pooling3 = MaxPooling2D((2, 2))(convolution3)
     
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(pooling3)
    convolution4 = BatchNormalization()(convolution4)
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution4)
    pooling4 = MaxPooling2D(pool_size=(2, 2))(convolution4)

    #Bottleneck at the base of the U-net 
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(pooling4)
    convolution5 = BatchNormalization()(convolution5)
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(convolution5)
    
    #Decoder where we are indicating to the model the precise location of the features 
    transconv6 = Conv2DTranspose(128, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution5)
    transconv6 = concatenate([transconv6, convolution4])
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(transconv6)
    convolution6 = BatchNormalization()(convolution6)
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution6)
     
    transconv7 = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution6)
    transconv7 = concatenate([transconv7, convolution3])
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(transconv7)
    convolution7 = BatchNormalization()(convolution7)
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution7)
     
    transconv8 = Conv2DTranspose(32, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution7)
    transconv8 = concatenate([transconv8, convolution2])
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(transconv8)
    convolution8 = BatchNormalization()(convolution8)
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution8)
     
    transconv9 = Conv2DTranspose(16, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution8)
    transconv9 = concatenate([transconv9, convolution1])
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(transconv9)
    convolution9 = BatchNormalization()(convolution9)
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution9)
     
    outputs = Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(convolution9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    
    iou = BinaryIoU()
    
    #loss options include: binary_crossentropy, IoU Loss (Jaccard Index), dice loss
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', iou]) 
    
    model.summary()
    
    return model

In [101]:
checkpoint_filepath = '../tmp/checkpoint'
es = EarlyStopping(patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,
                             save_weights_only=True,
                             monitor='val_loss', restore_best_weights=True)

In [102]:
model = unet_model(img_height=256, img_width=256, channels=3)

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_26 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_254 (Conv2D)            (None, 256, 256, 16  448         ['input_26[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_121 (Batch  (None, 256, 256, 16  64         ['conv2d_254[1][0]']             
 Normalization)                 )                                                          

In [103]:
model.fit(dataset, epochs = 50, batch_size=32, callbacks=[es], verbose=1)

Epoch 1/50
262/262 [==============================] - 85s 317ms/step - loss: 0.2750 - accuracy: 0.8877 - binary_io_u: 0.6609
Epoch 2/50
262/262 [==============================] - 79s 302ms/step - loss: 0.2064 - accuracy: 0.9175 - binary_io_u: 0.7375
Epoch 3/50
262/262 [==============================] - 79s 302ms/step - loss: 0.1823 - accuracy: 0.9279 - binary_io_u: 0.7681
Epoch 4/50
108/262 [===========>..................] - ETA: 46s - loss: 0.1621 - accuracy: 0.9365 - binary_io_u: 0.7910

KeyboardInterrupt: 

In [73]:
!pip freeze | grep tensorflow

tensorflow==2.8.3
tensorflow-cloud==0.1.16
tensorflow-datasets==4.7.0
tensorflow-estimator==2.8.0
tensorflow-hub==0.12.0
tensorflow-io==0.27.0
tensorflow-io-gcs-filesystem==0.27.0
tensorflow-metadata==1.8.0
tensorflow-probability==0.18.0
tensorflow-serving-api==2.8.3
tensorflow-transform==1.8.0
